# quake 3


## file management



### managing zips/pk3s



#### find pk3 files in zips?


In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var {listInProject} = importer.import('list project files')

function readZip(zipFile, cb) {
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        // Take a look at the files
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        var pk3s = Object.values(zip.entries())
            .filter(entry => entry.name.match(/\.pk3/i))
        
        return importer.runAllPromises(pk3s.map((pk3, i) => new Promise((resolve) => {
            var pk3Path = '/Users/briancullinan/planet_quake_data/bestmaps/' + path.basename(pk3.name);
            if(!fs.existsSync(pk3Path)) {
                zip.extract(pk3.name, pk3Path, err => {
                    if(err) {
                        console.log('Extract error ' + err)
                        return resolve()
                    }
                    console.log('Extracted ' + pk3.name);
                    resolve(pk3Path)
                });
            } else {
                resolve()
            }
        }))).then(() => {
            zip.close()
            cb()
        })
    });
    
    zip.on('error', err => { cb() });
}

function extractAll(root) {
    var results = listInProject(root || '/Users/briancullinan/planet_quake_resources', '**/*.zip')
    console.log(results)
    return importer.runAllPromises(results.map(r => {
        return (resolve) => {
            try {
                readZip(r, resolve)
            } catch (e) {
                if(e.message.includes('Bad archive')) {}
                else { console.log(e) }
                resolve()
            }
        }
    })).then(results => results.filter(r => r))
}

module.exports = extractAll;


#### find only bsps in map paks?



In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var {listInProject} = importer.import('list project files')


function readPak(zipFile, cb) {
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        for (const entry of Object.values(zip.entries())) {
            if(entry.name.includes('.bsp')) {
                console.log(entry.name)
                /*var pk3Path = '/Applications/ioquake3/bestmaps/' + path.basename(pk3.name);
                if(!fs.existsSync(pk3Path)) {
                    zip.extract(pk3.name, pk3Path, err => {
                        console.log((err ? 'Extract error ' : 'Extracted ') + pk3.name);
                        zip.close();
                        cb(zipFile);
                    });
                    return;
                }*/
            }
            if(entry.name.includes('levelshots/') && entry.name !== 'levelshots/') {
                var levelPath = '/Applications/ioquake3/bestmaps/levelshots/' + path.basename(entry.name)
                if(!fs.existsSync(levelPath)) {
                    zip.extract(entry.name, levelPath, err => {
                        console.log((err ? 'Extract error ' : 'Extracted ') + entry.name);
                        zip.close();
                        cb(zipFile);
                    });
                    return;
                }
            }
        }
        zip.close()
        cb();
    });
    
    zip.on('error', err => { cb() });
}


function extractPaks() {
    var results = listInProject('/Applications/ioquake3/bestmaps', '**/*.pk3')
    console.log(results)
    return importer.runAllPromises(results.map(r => {
        return (resolve) => {
            try {
                readPak(r, resolve)
            } catch (e) {
                if(e.message.includes('Bad archive')) {}
                else { console.log(e) }
                resolve()
            }
        }
    })).then(results => results.filter(r => r))
}


module.exports = extractPaks;


#### list bsps in a pak?

For that annoying problem when you download a map pk3 and it's a different name than the map.


In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');

function listBsps(zipFile) {
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        for (const entry of Object.values(zip.entries())) {
            if(entry.name.includes('.bsp')) {
                console.log(entry.name)
            }
        }
    });
    
    zip.on('error', err => {  });
}

module.exports = listBsps;


#### unpack entire folder of pk3s?

unpack pk3s?


In [ ]:
var StreamZip = require('node-stream-zip');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var {listInProject} = importer.import('list project files')
var mkdirpSync = importer.import('mkdirp')

function readPak(zipFile, cb, outdir) {
    if(!outdir) {
        outdir = '/Users/briancullinan/.q3a/unpacked/' + path.basename(zipFile) + 'dir'
    }
    const zip = new StreamZip({
        file: zipFile,
        storeEntries: true
    });

    zip.on('ready', () => {
        console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(zipFile));
        var index = Object.values(zip.entries())
        console.log(index.map(e => e.name))
        return Promise.all(index.map(entry => {
            var levelPath = path.join(outdir, entry.name)
            if(fs.existsSync(levelPath)) return Promise.resolve()
            mkdirpSync(path.dirname(levelPath))
            return new Promise(resolve => {
                zip.extract(entry.name, levelPath, err => {
                    console.log((err ? 'Extract error ' : 'Extracted ') + entry.name);
                    resolve(zipFile);
                });
            })
        })).then(() => {
            zip.close();
            cb()
        })
    });
    
    zip.on('error', err => { cb() });
}


function extractPaks(inpaks, outdir) {
    var results = listInProject(path.dirname(inpaks), '**/' + path.basename(inpaks))
    results.sort()
    console.log(results)
    return importer.runAllPromises(results.map(r => {
        return (resolve) => {
            try {
                readPak(r, resolve, outdir)
            } catch (e) {
                if(e.message.includes('Bad archive')) {}
                else { console.log(e) }
                resolve()
            }
        }
    })).then(results => results.filter(r => r))
}


module.exports = extractPaks;


### finding models and textures



### make indexes

look for some pk3dirs in the path, then make an index for every map file



#### the code

make pk3 indexes?


In [ ]:
var fs = require('fs')
var os = require('os')
var path = require('path')
var glob = require('glob')

var DEFAULT_PATH = path.join(process.env.HOME || process.env.HOMEPATH 
  || process.env.USERPROFILE || os.tmpdir(), '/.quake3/bestmaps-cc')

async function makePk3MapIndex(searchPath, prefixPath) {
    if(typeof searchPath == 'undefined' || !searchPath) {
        searchPath = DEFAULT_PATH
    }
    if(typeof prefixPath == 'undefined') {
        prefixPath = '/base/baseq3-cc'
    } else if (!prefixPath) {
        prefixPath = '/'
    }
    // TODO: find files in dir based on alt-filetypes and names listed in pk3 file,
    //   so a combined directory of files, but separate indexes
    var pk3dirs = glob.sync('**/*.pk3dir/', {
        nodir: false, cwd: searchPath, nocase: true
    })
    var allMaps = {}
    pk3dirs.forEach(dir => {
        var pk3path = path.join(searchPath, dir)
        var pk3files = glob.sync('**/*', {
            nodir: false, cwd: pk3path, nocase: true
        })
        var maps = pk3files.filter(file => file.match(/\.bsp$/i))
        var pk3Key = path.join(prefixPath, dir).toLowerCase()
        var initial = {}
        allMaps[pk3Key] = initial[pk3Key] = {
            name: path.join('/', dir).replace(/\/$/ig, '')
        }
        var manifest = pk3files.map(file => {
            var stat = fs.statSync(path.join(searchPath, dir, file))
            return stat.isDirectory() ? ({
                name: path.join('/', dir, file).replace(/\/$/ig, ''),
            }) : ({
                name: path.join('/', dir, file),
                size: stat.size
            })
        }).reduce((obj, o) => {
            var key = path.join(prefixPath, o.name).toLowerCase()
                + (typeof o.size == 'undefined' ? '/' : '')
            obj[key] = o
            return obj
        }, initial)
        var manifestJson = JSON.stringify(manifest, null, 2)
        maps.forEach(map => {
            var mapName = path.basename(map).toLowerCase().replace(/\.bsp/i, '')
            var outIndexFile = path.join(searchPath, 'index-' + mapName + '.json')
            var key = path.join(prefixPath, path.basename(pk3path), map).toLowerCase()
            fs.writeFileSync(outIndexFile, manifestJson)
            allMaps[key] = {
                name: path.join('/', dir, map),
                size: fs.statSync(path.join(searchPath, dir, map)).size
            }
        })
    })
    return JSON.stringify(allMaps, null, 2)
}

module.exports = makePk3MapIndex


### make checksums




#### the code

crc checksum file?



In [ ]:
var fs = require('fs')
var path = require('path')
var StreamZip = require('node-stream-zip')

/* NOTE: This code makes no attempt to be fast!

It assumes that an int is at least 32 bits long
*/

var m

function F(X,Y,Z) {return (((X)&(Y)) | ((~(X))&(Z)))}
function G(X,Y,Z) {return (((X)&(Y)) | ((X)&(Z)) | ((Y)&(Z)))}
function H(X,Y,Z) {return ((X)^(Y)^(Z))}
function lshift(x,s) {return (((x)<<(s)) | ((x)>>>(32-(s))))}

/* this applies md4 to 64 byte chunks */
function mdfour64(M)
{
    var AA, BB, CC, DD
    var X = new Uint32Array(16)
    var A, B, C, D

    for (var j=0;j<16;j++)
        X[j] = M[j];

    function ROUND1(a,b,c,d,k,s) { return lshift(a + F(b,c,d) + X[k], s) }
    function ROUND2(a,b,c,d,k,s) { return lshift(a + G(b,c,d) + X[k] + 0x5A827999,s) }
    function ROUND3(a,b,c,d,k,s) { return lshift(a + H(b,c,d) + X[k] + 0x6ED9EBA1,s) }

 	A = m.A; B = m.B; C = m.C; D = m.D;
	AA = A; BB = B; CC = C; DD = D;

    A = ROUND1(A,B,C,D,  0,  3)
    D = ROUND1(D,A,B,C,  1,  7)
    C = ROUND1(C,D,A,B,  2, 11)
    B = ROUND1(B,C,D,A,  3, 19)
   
    A = ROUND1(A,B,C,D,  4,  3)
    D = ROUND1(D,A,B,C,  5,  7)
    C = ROUND1(C,D,A,B,  6, 11)
    B = ROUND1(B,C,D,A,  7, 19)
    
    A = ROUND1(A,B,C,D,  8,  3)
    D = ROUND1(D,A,B,C,  9,  7)
    C = ROUND1(C,D,A,B, 10, 11)
    B = ROUND1(B,C,D,A, 11, 19)
    
    A = ROUND1(A,B,C,D, 12,  3)
    D = ROUND1(D,A,B,C, 13,  7)
    C = ROUND1(C,D,A,B, 14, 11)
    B = ROUND1(B,C,D,A, 15, 19)

    
    A = ROUND2(A,B,C,D,  0,  3)
    D = ROUND2(D,A,B,C,  4,  5)
    C = ROUND2(C,D,A,B,  8,  9)
    B = ROUND2(B,C,D,A, 12, 13)
    
    A = ROUND2(A,B,C,D,  1,  3)
    D = ROUND2(D,A,B,C,  5,  5)
    C = ROUND2(C,D,A,B,  9,  9)
    B = ROUND2(B,C,D,A, 13, 13)
    
    A = ROUND2(A,B,C,D,  2,  3)
    D = ROUND2(D,A,B,C,  6,  5)
    C = ROUND2(C,D,A,B, 10,  9)
    B = ROUND2(B,C,D,A, 14, 13)
    
    A = ROUND2(A,B,C,D,  3,  3)
    D = ROUND2(D,A,B,C,  7,  5)
    C = ROUND2(C,D,A,B, 11,  9)
    B = ROUND2(B,C,D,A, 15, 13)

    
    A = ROUND3(A,B,C,D,  0,  3)
    D = ROUND3(D,A,B,C,  8,  9)
    C = ROUND3(C,D,A,B,  4, 11)
    B = ROUND3(B,C,D,A, 12, 15)
    
    A = ROUND3(A,B,C,D,  2,  3)
    D = ROUND3(D,A,B,C, 10,  9)
    C = ROUND3(C,D,A,B,  6, 11)
    B = ROUND3(B,C,D,A, 14, 15)
    
    A = ROUND3(A,B,C,D,  1,  3)
    D = ROUND3(D,A,B,C,  9,  9)
    C = ROUND3(C,D,A,B,  5, 11)
    B = ROUND3(B,C,D,A, 13, 15)
    
    A = ROUND3(A,B,C,D,  3,  3)
    D = ROUND3(D,A,B,C, 11,  9)
    C = ROUND3(C,D,A,B,  7, 11)
    B = ROUND3(B,C,D,A, 15, 15)

    A += AA; B += BB; C += CC; D += DD;

    for (var j=0;j<16;j++)
        X[j] = 0;

    m.A = A; m.B = B; m.C = C; m.D = D;
}

function copy64(M, input)
{
    for (var i=0;i<16;i++)
        M[i] =
            (input[i*4+3] << 24) |
            (input[i*4+2] << 16) |
            (input[i*4+1] << 8) |
            (input[i*4+0] << 0)
}

function mdfour_begin(md)
{
    md.A = 0x67452301
    md.B = 0xefcdab89
    md.C = 0x98badcfe
    md.D = 0x10325476
    md.totalN = 0
}


function mdfour_tail(input, n)
{
    var buf = new Uint8Array(128)
    var M = new Uint32Array(16)
    var b

    m.totalN += n

    b = m.totalN * 8

    if (n) for(var i=0;i<n;i++) buf[i] = input[i]
    buf[n] = 0x80

    if (n <= 55) {
        buf[56] = (b&0xFF)
        buf[57] = (b>>8)&0xFF
        buf[58] = (b>>16)&0xFF
        buf[59] = (b>>24)&0xFF
        copy64(M, buf)
        mdfour64(M)
    } else {
        buf[120] = (b&0xFF)
        buf[121] = (b>>8)&0xFF
        buf[122] = (b>>16)&0xFF
        buf[123] = (b>>24)&0xFF
        copy64(M, buf)
        mdfour64(M)
        copy64(M, buf.slice(64))
        mdfour64(M)
    }
}

function mdfour_update(md, input, n)
{
    var M = new Uint32Array(16)

    m = md

    if (n == 0) mdfour_tail(input, n)

    while (n - m.totalN >= 64) {
        copy64(M, input.slice(m.totalN, m.totalN + 64))
        mdfour64(M)
        m.totalN += 64
    }

    mdfour_tail(input.slice(m.totalN, m.totalN + 64), n - m.totalN)
}


function mdfour_result(md, out)
{
    out[0] = md.A
    out[1] = md.B
    out[2] = md.C
    out[3] = md.D
}

function mdfour(out, input, n)
{
    var md = {
        A: 0,
        B: 0,
        C: 0,
        D: 0,
        totalN: 0
    }
    mdfour_begin(md)
    mdfour_update(md, input, n)
    mdfour_result(md, out)
}

async function checksumZip(file) {
    var digest = new Uint32Array(4)
    const zip = new StreamZip({
      file: file,
      storeEntries: true
    })
    var index = await new Promise(resolve => {
        var skipped = 0
        zip.on('ready', async () => {
          console.log('Entries read: ' + zip.entriesCount + ' ' + path.basename(file))
          resolve(Object.values(zip.entries()))
        })
        zip.on('error', resolve)
    })
    var contents = [0]
    var j = 1
    for(var i = 0; i < index.length; i++) {
        var entry = index[i]
        if((entry.method != 0 && entry.method != 8) || entry.size === 0)
            continue
        contents[j++] = entry.crc
    }
    var headers = new Uint8Array(Uint32Array.from(contents).buffer)
    process.stdout.write(JSON.stringify(contents))
    mdfour(digest, headers.slice(4, j * 4), j * 4 - 4)
    var unsigned = new Uint32Array(1)
    unsigned[0] = digest[0] ^ digest[1] ^ digest[2] ^ digest[3]
    return unsigned
}

module.exports = checksumZip


#### test crc file?



In [ ]:
var importer = require('../Core')
var mdfour = importer.import('crc checksum file')

var TEST_PK3 = '/Applications/ioquake3/baseq3/pak0.pk3'

async function testCrcFile() {
    return await checksumZip(TEST_PK3)
}

/*
1566731103,
298122907,
412165236,
2991495316,
1197932710,
4087071573,
3709064859,
908855077,
977125798
*/

module.exports = testCrcFile


### converting graphics and audio

Pretty tired of messing around with bash conversions. Call imagemagick commands from Node because strings and file names are easier to manipulate in JS.


#### convert images that don't have alpha

convert nonalpha?


In [ ]:
var path = require('path')
var fs = require('fs')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var execCmd = importer.import('spawn child process')
var mkdirpSync = importer.import('mkdirp')
var { chext, chroot } = importer.import('changing file names')
var {imageTypes} = importer.import('quake 3 file whitelist')

async function convertNonAlpha(root) {
    var result = []
    var output = path.join(path.dirname(root), path.basename(root) + '-converted')
    var files = await glob(imageTypes.map(a => '**/*' + a), root)
    for(var i = 0; i < files.length; i++) {
        var outFile, inFile = files[i]
        var alphaCmd = await execCmd(`identify -format '%[opaque]' '${inFile}'`, {quiet: true})
        // if it is alpha
        if(alphaCmd[0].localeCompare('False') === 0) {
            // convert everything else to png to support transparency
            outFile = chroot(chext(inFile, '.png'), root, output)
        } else {
            // if a jpg already exists, use that file for convert
            if(fs.existsSync(chext(inFile, '.jpg'))) {
                inFile = chext(inFile, '.jpg')
            }
            // transfer low quality jpeg instead
            outFile = chroot(chext(inFile, '.jpg'), root, output)
        }
        mkdirpSync(path.dirname(outFile))
        // convert, baseq3 already includes jpg
        await execCmd(`convert -strip -interlace Plane -sampling-factor 4:2:0 -quality 50% "${inFile}" "${outFile}"`, {quiet: true})
        result.push(outFile)
    }
    return result
}


module.exports = convertNonAlpha

#### convert quake 3 audio?



In [ ]:
var path = require('path')
var fs = require('fs')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var execCmd = importer.import('spawn child process')
var mkdirpSync = importer.import('mkdirp')
var { chext, chroot } = importer.import('changing file names')
var {audioTypes} = importer.import('quake 3 file whitelist')

async function convertAudio(root) {
    var result = []
    var output = path.join(path.dirname(root), path.basename(root) + '-converted')
    
    var files = await glob(audioTypes.map(a => '**/*' + a), root)
    for(var i = 0; i < files.length; i++) {
        var outFile, inFile = files[i]
        outFile = chroot(chext(inFile, '.opus'), root, output)
        mkdirpSync(path.dirname(outFile))
        await execCmd(`opusenc --bitrate 24 --vbr "${inFile}" "${outFile}"`, {quiet: false})
        result.push(outFile)
    }
    return result
}

module.exports = convertAudio


#### copy quake 3 script files?


In [ ]:
var path = require('path')
var fs = require('fs')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var mkdirpSync = importer.import('mkdirp')
var { chext, chroot } = importer.import('changing file names')
var {fileTypes} = importer.import('quake 3 file whitelist')

async function convertScripts(root) {
    var result = []
    var output = path.join(path.dirname(root), path.basename(root) + '-converted')
    var files = await glob(fileTypes.map(a => '**/*' + a), root)
    for(var i = 0; i < files.length; i++) {
        var inFile = files[i]
        var outFile = chroot(inFile, root, output)
        if(!fs.existsSync(outFile)) {
            mkdirpSync(path.dirname(outFile))
            await fs.copyFileSync(inFile, outFile)
            result.push(outFile)
        }
    }
    return result
}

module.exports = convertScripts


#### whitelist for all quake 3 file types

quake 3 file whitelist?


TODO: scan lots of mods and file files that aren't in the list

TODO: scan the quake 3 code and look for extensions, particularly Unvanquished which adds lots of file types for compatibility.


In [ ]:

var imageTypes = [
    '.png',
    '.jpg',
    '.jpeg',
    '.tga',
    '.gif',
    '.pcx',
    '.webp',
]

var audioTypes = [
    '.opus',
    '.wav',
    '.mp3',
]

var sourceFiles = [
    '.map',
    '.scc',
]

var fileTypes = [
    '.cfg',
    '.qvm',
    '.bot',
    '.txt',
    '.bsp',
    '.aas',
    '.md3',
    '.md5',
    '.shader',
    '.skin',
    '.pk3',
    '.c',
    '.h',
    '.config',
    '.menu'
]


var knownDirs = [
    'scripts',
    'botfiles',
	'fonts',
	'gfx',
    'hud',
	'icons',
	'include',
	'menu',
	'models',
	'music',
	'powerups',  // powerup shaders
	'sprites',
	'sound',
	'ui',
    'maps',
    'textures',
    
]

module.exports = {
    imageTypes,
    audioTypes,
    sourceFiles,
    fileTypes,
    knownDirs,
}



#### test pack creation with results from ]\imagelist

quake 3 menu image list?


In [ ]:
var fs = require('fs')
var importer = require('../Core')
var { chext, chroot } = importer.import('changing file names')
var {glob} = importer.import('glob files')
var mkdirpSync = importer.import('mkdirp')

var imagelist = `
  68:   32x  32 RGB        4kb gfx/misc/flare.tga
  69:   32x  32 RGB        4kb gfx/misc/sun.tga
  70:  256x 256 RGBA     256kb gfx/2d/bigchars.tga
  71:  128x 128 RGBA      64kb gfx/misc/console01.tga
  72:  128x 128 RGB       48kb gfx/misc/console02.tga
  73:  256x 256 RGBA     256kb menu/art/font1_prop.tga
  74:  256x 256 RGBA     256kb menu/art/font1_prop_glo.tga
  75:  256x 256 RGBA     256kb menu/art/font2_prop.tga
  76:   32x  32 RGBA       4kb menu/art/3_cursor2
  77:   16x  16 RGBA    1024b  menu/art/switch_on
  78:   16x  16 RGBA    1024b  menu/art/switch_off
  79:  512x 512 RGB      768kb textures/sfx/logo512.tga
  80:    8x   8 RGB      192b  gfx/colors/black.tga
  81:  128x  16 RGBA       8kb menu/art/slider2
  82:   16x  32 RGBA       2kb menu/art/sliderbutt_0
  83:   16x  32 RGBA       2kb menu/art/sliderbutt_1
  84:  128x 128 RGB       48kb levelshots/q3tourney1.tga
  85:  256x 256 RGB      288kb textures/sfx/detail.tga
  86:  512x 512 RGBA    1024kb _fatlightmap0
  87:  128x 128 RGB       72kb textures/gothic_block/blocks18b
  88:  128x 128 RGB       72kb textures/base_wall/bluemetal2
  89:   64x  64 RGB       18kb textures/gothic_trim/metalsupsolid
  90:   64x  64 RGB       18kb textures/base_trim/dirty_pewter
  91:   64x  64 RGB       18kb textures/base_wall/greenmet2b
  92:  128x  32 RGB       18kb textures/gothic_trim/km_arena1tower4_a
  93:  128x 128 RGB       72kb textures/gothic_block/blocks11b
  94:  128x 128 RGB       72kb textures/gothic_trim/metalsupport4c
  95:  256x 256 RGB      288kb textures/gothic_door/archxiandm1dblack
  96:  128x 128 RGB       72kb textures/gothic_door/xian_tourneyarch_inside2
  97:  256x 256 RGB      288kb textures/gothic_door/archxiandm1b
  98:  128x 128 RGB       72kb textures/gothic_block/blocks18c
  99:  128x 128 RGB       72kb textures/gothic_trim/pitted_rust
 100:  128x 128 RGB       72kb textures/gothic_trim/metalsupport4
 101:  128x 128 RGB       72kb textures/gothic_trim/metalsupport4d
 102:  128x  64 RGB       36kb textures/gothic_trim/border11
 103:   64x  64 RGB       18kb textures/sfx/tesla1.tga
 104:   64x  64 RGB       18kb textures/sfx/electricslime.tga
 105:  128x 128 RGBA      96kb textures/sfx/cabletest2.tga
 106:   32x 128 RGB       18kb textures/sfx/hologirl.tga
 107:  128x 128 RGBA      96kb textures/base_wall/atech1_alpha.tga
 108:  128x 128 RGB       72kb textures/gothic_cath/smalldoor_xiandm9a
 109:  128x 128 RGB       72kb textures/gothic_cath/smalldoor_xiandm9i
 110:  128x  64 RGB       36kb textures/base_trim/spiderbit3
 111:  128x 128 RGB       72kb env/xnight2_rt.tga
 112:  128x 128 RGB       72kb env/xnight2_bk.tga
 113:  128x 128 RGB       72kb env/xnight2_lf.tga
 114:  128x 128 RGB       72kb env/xnight2_ft.tga
 115:  128x 128 RGB       72kb env/xnight2_up.tga
 116:  128x 128 RGB       72kb env/xnight2_dn.tga
 117:  128x 128 RGB       72kb textures/sfx/demonltblackfinal.tga
 118:  128x 128 RGB       72kb textures/sfx/demonltblackfinal_glow2.tga
 119:   16x  16 RGB        1kb textures/base_light/ceil1_39.tga
 120:   16x  16 RGB        1kb textures/base_light/ceil1_39.blend.tga
 121:   32x  32 RGB        4kb textures/gothic_light/gothic_light3.tga
 122:   32x  32 RGB        4kb textures/gothic_light/gothic_light2_blend.tga
 123:  128x 128 RGB       72kb textures/gothic_floor/pent_metalbridge06.tga
 124:  128x 128 RGB       72kb textures/gothic_floor/pent_glow.tga
 125:   64x  64 RGB       18kb textures/gothic_ceiling/stucco7top
 126:  128x 128 RGB       72kb textures/gothic_trim/pitted_rust3
 127:   16x  16 RGB        1kb textures/base_light/ceil1_38.tga
 128:   16x  16 RGB        1kb textures/base_light/ceil1_38.blend.tga
 129:  128x 128 RGB       72kb textures/base_floor/metalbridge06
 130:  128x 128 RGBA      96kb textures/base_floor/proto_rustygrate.tga
 131:   64x  64 RGB       18kb textures/sfx/jacobs_x.tga
 132:   16x 128 RGB        9kb textures/sfx/beam_1.tga
 133:  128x 128 RGB       72kb models/mapobjects/Bitch/bitch
 134:   64x  64 RGB       18kb models/mapobjects/bitch/orb.tga
 135:   16x  32 RGB        2kb models/mapobjects/bitch/forearm.tga
 136:   16x  32 RGB        2kb models/mapobjects/bitch/forearm01.tga
 137:   16x  32 RGB        2kb models/mapobjects/bitch/forearm02.tga
 138:  128x 128 RGB       72kb models/mapobjects/pitted_rust_ps.tga
 139:   64x  64 RGB       18kb models/mapobjects/flares/electric.tga
 140:   64x 128 RGB       36kb textures/sfx/flame1.tga
 141:   64x 128 RGB       36kb textures/sfx/flame2.tga
 142:   64x 128 RGB       36kb textures/sfx/flame3.tga
 143:   64x 128 RGB       36kb textures/sfx/flame4.tga
 144:   64x 128 RGB       36kb textures/sfx/flame5.tga
 145:   64x 128 RGB       36kb textures/sfx/flame6.tga
 146:   64x 128 RGB       36kb textures/sfx/flame7.tga
 147:   64x 128 RGB       36kb textures/sfx/flame8.tga
 148:   64x 128 RGB       36kb textures/sfx/flameball.tga
 149:   16x  32 RGB        2kb models/mapobjects/gratelamp/gratetorch2
 150:   16x  32 RGBA       3kb models/mapobjects/gratelamp/gratetorch2b.tga
 151:   64x  64 RGB       18kb textures/liquids/lavahell.tga
 152:   32x  32 RGBA       6kb gfx/2d/numbers/zero_32b.tga
 153:   32x  32 RGBA       6kb gfx/2d/numbers/one_32b.tga
 154:   32x  32 RGBA       6kb gfx/2d/numbers/two_32b.tga
 155:   32x  32 RGBA       6kb gfx/2d/numbers/three_32b.tga
 156:   32x  32 RGBA       6kb gfx/2d/numbers/four_32b.tga
 157:   32x  32 RGBA       6kb gfx/2d/numbers/five_32b.tga
 158:   32x  32 RGBA       6kb gfx/2d/numbers/six_32b.tga
 159:   32x  32 RGBA       6kb gfx/2d/numbers/seven_32b.tga
 160:   32x  32 RGBA       6kb gfx/2d/numbers/eight_32b.tga
 161:   32x  32 RGBA       6kb gfx/2d/numbers/nine_32b.tga
 162:   32x  32 RGBA       6kb gfx/2d/numbers/minus_32b.tga
 163:   16x  16 RGBA       1kb menu/art/skill1.tga
 164:   16x  16 RGBA       1kb menu/art/skill2.tga
 165:   16x  16 RGBA       1kb menu/art/skill3.tga
 166:   16x  16 RGBA       1kb menu/art/skill4.tga
 167:   16x  16 RGBA       1kb menu/art/skill5.tga
 168:   32x  32 RGBA       6kb gfx/damage/blood_screen.tga
 169:   32x  32 RGBA       4kb gfx/2d/defer.tga
 170:  128x  32 RGBA      16kb menu/tab/name.tga
 171:  128x  32 RGBA      16kb menu/tab/ping.tga
 172:  128x  32 RGBA      16kb menu/tab/score.tga
 173:  128x  32 RGBA      16kb menu/tab/time.tga
 174:   64x  64 RGBA      24kb gfx/misc/smokepuff3.tga
 175:   64x  64 RGBA      24kb gfx/misc/smokepuffragepro.tga
 176:   64x  64 RGBA      24kb gfx/misc/smokepuff2b.tga
 177:   32x  32 RGB        4kb sprites/plasmaa.tga
 178:   64x  64 RGBA      24kb gfx/damage/blood_spurt.tga
 179:   32x  32 RGB        4kb gfx/2d/lag.tga
 180:   32x  32 RGB        4kb gfx/2d/net.tga
 181:   16x  16 RGBA       1kb sprites/bubble.tga
 182:    8x   8 RGB      288b  gfx/misc/tracer2.tga
 183:   32x  32 RGBA       6kb gfx/2d/select.tga
 184:   16x  16 RGBA       1kb gfx/2d/crosshaira
 185:   32x  32 RGBA       6kb gfx/2d/crosshairb.tga
 186:   32x  32 RGBA       6kb gfx/2d/crosshairc.tga
 187:   32x  32 RGBA       6kb gfx/2d/crosshaird.tga
 188:   32x  32 RGBA       6kb gfx/2d/crosshaire.tga
 189:   32x  32 RGBA       6kb gfx/2d/crosshairf.tga
 190:   32x  32 RGBA       6kb gfx/2d/crosshairg.tga
 191:   32x  32 RGBA       6kb gfx/2d/crosshairh.tga
 192:   32x  32 RGBA       6kb gfx/2d/crosshairi.tga
 193:   32x  32 RGBA       6kb gfx/2d/crosshairj.tga
 194:   32x  32 RGB        4kb gfx/2d/backtile
 195:   16x  16 RGBA       1kb icons/noammo
 196:   32x  32 RGB        4kb textures/effects/quadmap2.tga
 197:   32x  32 RGB        4kb textures/effects/envmapgold2.tga
 198:   32x  32 RGB        4kb textures/effects/invismap.tga
 199:   32x  32 RGB        4kb textures/effects/regenmap2.tga
 200:   64x  64 RGB       18kb textures/sfx/specular.tga
 201:   64x  64 RGBA      24kb models/powerups/armor/newyellow.tga
 202:   64x  32 RGB        9kb models/powerups/armor/energy_yel3.tga
 203:   32x  32 RGBA       6kb icons/iconr_yellow.tga
 204:   32x  32 RGB        4kb models/weapons2/shells/M_shell.tga
 205:   32x  32 RGB        4kb models/weapons2/shells/s_shell.tga
 206:  128x 128 RGB       72kb models/gibs/gibs.tga
 207:   32x  32 RGBA       6kb sprites/balloon4.tga
 208:   32x  32 RGBA       6kb models/weaphits/blood201.tga
 209:   32x  32 RGBA       6kb models/weaphits/blood202.tga
 210:   32x  32 RGBA       6kb models/weaphits/blood203.tga
 211:   32x  32 RGBA       6kb models/weaphits/blood204.tga
 212:   32x  32 RGBA       6kb models/weaphits/blood205.tga
 213:   32x  32 RGB        4kb models/weaphits/bullet1.tga
 214:   32x  32 RGB        4kb models/weaphits/bullet2.tga
 215:   32x  32 RGB        4kb models/weaphits/bullet3.tga
 216:   32x  32 RGBA       6kb models/weaphits/ring01_1.tga
 217:   64x  64 RGB       18kb models/weaphits/newexplo1.tga
 218:  128x 128 RGB       72kb gfx/misc/teleportEffect2.tga
 219:   64x  64 RGBA      24kb menu/medals/medal_impressive.tga
 220:   64x  64 RGBA      24kb menu/medals/medal_excellent.tga
 221:   64x  64 RGBA      24kb menu/medals/medal_gauntlet.tga
 222:   64x  64 RGBA      24kb menu/medals/medal_defend.tga
 223:   64x  64 RGBA      24kb menu/medals/medal_assist.tga
 224:   64x  64 RGBA      24kb menu/medals/medal_capture.tga
 225:   32x  32 RGBA       6kb icons/iconh_yellow.tga
 226:   32x  32 RGB        4kb textures/effects/envmapyel.tga
 227:   64x  64 RGB       18kb textures/effects/tinfx2b.tga
 228:   32x  32 RGBA       6kb icons/iconh_red.tga
 229:   32x  32 RGB        4kb textures/effects/envmapgold.tga
 230:   32x  32 RGB        4kb textures/sfx/kenelectric.tga
 231:   32x  32 RGBA       6kb icons/iconw_gauntlet.tga
 232:  128x 128 RGB       72kb models/weapons2/gauntlet/gauntlet1.tga
 233:   64x  64 RGB       18kb models/weapons2/gauntlet/gauntlet3.tga
 234:   64x  64 RGB       18kb models/weapons2/gauntlet/gauntlet4.tga
 235:   64x  64 RGBA      24kb models/weapons2/gauntlet/gauntlet2.tga
 236:   32x  32 RGBA       6kb icons/iconw_shotgun.tga
 237:  128x 128 RGB       72kb models/weapons2/shotgun/shotgun.tga
 238:   32x   2 RGB      288b  models/weapons2/shotgun/shotgun_laser.tga
 239:   32x  32 RGB        4kb models/powerups/ammo/shotammo2.tga
 240:   64x  64 RGB       18kb textures/effects/tinfx2c.tga
 241:   64x  64 RGBA      24kb models/powerups/ammo/shotammo.tga
 242:   64x  64 RGB       18kb models/weapons2/shotgun/f_shotgun.tga
 243:   32x  32 RGBA       6kb icons/iconw_machinegun.tga
 244:  128x 128 RGB       72kb models/weapons2/machinegun/machinegun.tga
 245:   32x  32 RGBA       6kb models/powerups/ammo/machammo2.tga
 246:   64x  64 RGB       18kb textures/effects/tinfx2d.tga
 247:   64x  64 RGBA      24kb models/powerups/ammo/machammo.tga
 248:   64x  64 RGB       18kb models/weapons2/machinegun/f_machinegun.tga
 249:   32x  32 RGBA       6kb icons/iconw_rocket.tga
 250:  128x 128 RGB       72kb models/weapons2/rocketl/rocketl.TGA
 251:   32x  32 RGB        4kb models/weapons2/rocketl/rocketl2.tga
 252:   32x  32 RGB        4kb models/powerups/ammo/rockammo2.tga
 253:   64x  64 RGBA      24kb models/powerups/ammo/rockammo.tga
 254:   64x  64 RGB       18kb models/weapons2/rocketl/f_rocketl.tga
 255:   32x  32 RGB        4kb models/ammo/rocket/rockfls1.tga
 256:   32x  32 RGB        4kb models/ammo/rocket/rockfls2.tga
 257:   32x  16 RGB        2kb models/ammo/rocket/rockflar.tga
 258:   32x  32 RGB        4kb models/ammo/rocket/rocket.tga
 259:   16x  16 RGB        1kb models/ammo/rocket/rocketfn.tga
 260:   32x  32 RGB        4kb models/weaphits/rlboom/rlboom_1.tga
 261:   32x  32 RGB        4kb models/weaphits/rlboom/rlboom_2.tga
 262:   32x  32 RGB        4kb models/weaphits/rlboom/rlboom_3.tga
 263:   32x  32 RGB        4kb models/weaphits/rlboom/rlboom_4.tga
 264:   32x  32 RGB        4kb models/weaphits/rlboom/rlboom_5.tga
 265:   32x  32 RGB        4kb models/weaphits/rlboom/rlboom_6.tga
 266:   32x  32 RGB        4kb models/weaphits/rlboom/rlboom_7.tga
 267:   32x  32 RGB        4kb models/weaphits/rlboom/rlboom_8.tga
 268:   32x  32 RGBA       6kb icons/icona_shotgun.tga
 269:   32x  32 RGBA       6kb icons/icona_machinegun.tga
 270:   32x  32 RGBA       6kb icons/icona_rocket.tga
 271:   32x  32 RGBA       6kb icons/quad.tga
 272:   32x  32 RGB        4kb textures/effects/envmapblue.tga
 273:   32x  32 RGBA       6kb icons/regen.tga
 274:   32x  32 RGB        4kb textures/effects/envmapred.tga
 275:    8x   8 RGB      288b  gfx/damage/bullet_mrk.tga
 276:   32x  32 RGB        4kb gfx/damage/burn_med_mrk.tga
 277:   32x  32 RGB        4kb gfx/damage/hole_lg_mrk.tga
 278:   32x  32 RGBA       6kb gfx/damage/plasma_mrk.tga
 279:   16x  16 RGBA       1kb gfx/damage/shadow.tga
 280:   32x  32 RGB        4kb sprites/splash.tga
 281:   64x  64 RGBA      24kb gfx/damage/blood_stain.tga
 282:   32x  32 RGB        4kb sprites/explode1/fs2-0000.tga
 283:   32x  32 RGB        4kb sprites/explode1/fs2-0001.tga
 284:   32x  32 RGB        4kb sprites/explode1/fs2-0002.tga
 285:   32x  32 RGB        4kb sprites/explode1/fs2-0003.tga
 286:   32x  32 RGB        4kb sprites/explode1/fs2-0004.tga
 287:   32x  32 RGB        4kb sprites/explode1/fs2-0005.tga
 288:   32x  32 RGB        4kb sprites/explode1/fs2-0006.tga
 289:   32x  32 RGB        4kb sprites/explode1/fs2-0007.tga
 290:   32x  32 RGB        4kb sprites/explode1/fs2-0008.tga
 291:   32x  32 RGB        4kb sprites/explode1/fs2-0009.tga
 292:   32x  32 RGB        4kb sprites/explode1/fs2-0010.tga
 293:   32x  32 RGB        4kb sprites/explode1/fs2-0011.tga
 294:   32x  32 RGB        4kb sprites/explode1/fs2-0012.tga
 295:   32x  32 RGB        4kb sprites/explode1/fs2-0013.tga
 296:   32x  32 RGB        4kb sprites/explode1/fs2-0014.tga
 297:   32x  32 RGB        4kb sprites/explode1/fs2-0015.tga
 298:   32x  32 RGB        4kb sprites/explode1/fs2-0016.tga
 299:   32x  32 RGB        4kb sprites/explode1/fs2-0017.tga
 300:   32x  32 RGB        4kb sprites/explode1/fs2-0018.tga
 301:   32x  32 RGB        4kb sprites/explode1/fs2-0019.tga
 302:   32x  32 RGB        4kb sprites/explode1/fs2-0020.tga
 303:   32x  32 RGB        4kb sprites/explode1/fs2-0021.tga
 304:   32x  32 RGB        4kb sprites/explode1/fs2-0022.tga
 305:   64x  64 RGBA      16kb models/players/sarge/icon_default.tga
 306:  128x 128 RGB       72kb models/players/sarge/band.tga
 307:    8x   4 RGB      144b  models/players/sarge/cigar.tga
 308:    8x   4 RGB      144b  models/players/sarge/cigar.glow.tga
`

var shaderlist = `
1     gen : gfx/2d/sunflare
1   E gen : gfx/2d/bigchars
1   E gen : white
2   E gen : console
1     gen : menu/art/font1_prop
1     gen : menu/art/font1_prop_glo
1     gen : menu/art/font2_prop
1     gen : menu/art/3_cursor2
1     gen : menu/art/switch_on
1     gen : menu/art/switch_off
1   E gen : menuback
1   E gen : menubacknologo
1     gen : menu/art/slider2
1     gen : menu/art/sliderbutt_0
1     gen : menu/art/sliderbutt_1
3   E gen : models/mapobjects/banner/q3banner02
3   E gen : models/mapobjects/banner/q3banner04
1     gen : menu/art/maps_select
1     gen : menu/art/maps_selected
1     gen : menu/art/narrow_0
1     gen : menu/art/narrow_1
1     gen : menu/art/unknownmap
1     gen : menu/art/level_complete1
1     gen : menu/art/level_complete2
1     gen : menu/art/level_complete3
1     gen : menu/art/level_complete4
1     gen : menu/art/level_complete5
1     gen : menu/art/back_0
1     gen : menu/art/back_1
1     gen : menu/art/fight_0
1     gen : menu/art/fight_1
1     gen : menu/art/reset_0
1     gen : menu/art/reset_1
1     gen : menu/art/skirmish_0
1     gen : menu/art/skirmish_1
1     gen : menu/medals/medal_accuracy
1     gen : menu/medals/medal_impressive
1     gen : menu/medals/medal_excellent
1     gen : menu/medals/medal_gauntlet
1     gen : menu/medals/medal_frags
1     gen : menu/medals/medal_victory
1     gen : models/players/sarge/icon_default
1     gen : levelshots/q3dm0
1     gen : models/players/crash/icon_default
`

async function getImages(root) {
    var result = []
    var output = path.join(path.dirname(root), path.basename(root) + '-menu')
    var images = imagelist
        .split('\n').map(l => l.split(/[\t\s]+/ig)[6])
        .filter(i => i)
        .map(i => '**/' + chext(i, '') + '*')
    var files = await glob(images, root)
    for(var i = 0; i < files.length; i++) {
        var outFile = chroot(files[i], root, output)
        mkdirpSync(path.dirname(outFile))
        fs.copyFileSync(files[i], outFile)
        result.push(outFile)
    }
    
    
    return result
}

module.exports = getImages


### TODO: organize textures, find overrides (accidental and intentional)

Does TrenchBroom really require everything be in 1 folder?


TODO: all world, object and player textures run through waifu2x and SFTGAN


## convert quake 2 map to quake 3


#### the code

convert quake 2 map to quake 3?


In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')
var replaceClasses = importer.import('replace known classes in map')
var replaceEntities = importer.import('replace entities in map')
var replaceTextures = importer.import('replace common textures in map')
var scaleMap = importer.import('scale quake map')
var {addSkybox} = importer.import('add skybox to map')
var {MAPS_HINTS, addHints} = importer.import('draw hints in map')

function convertMap(fileName) {
    var file;
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        file = fs.readFileSync(fileName).toString('utf-8')
    }
    
    file = replaceEntities(file)
    
    file = replaceClasses(file)
    
    // replace water with foggy water
    
    // replace message with target print
    
    // replace duplicate textures / don't interfere with basepack
    file = replaceTextures(file)
    
    // replace barrels with rocket launchers
    
    // replace animations and generate proper shaders
    // scale map 1.2x
    file = scaleMap(file, 1.15)
    
    //file = addSkybox(file)
    
    // scale lighting with q3map2 bsp options
    var newFile = fileName.substr(0, fileName.length - path.extname(fileName).length) + '.map'
    console.log(`writing ${newFile}`)
    fs.writeFileSync(newFile, file)
}

module.exports = convertMap;


#### scale quake map?



In [ ]:
var fs = require('fs')
var DIGITS = 100.0

function scaleMap(file, amount) {
    // get all brushes in map, leaf nodes with at least one vertex
    var brushes = importer.regexToArray(/\{[\s\S]*?\}/ig, file)
    
    // replace all brushes with scaled values
    brushes.forEach(b => {
        var newBrush = b
        newBrush = newBrush.replace(/\(((\s*[0-9\.-]+\s*)*)\)/ig, (str, $1) => {
            return '( ' + $1.trim().split(/\s+/ig)
                .map(n => Math.round((n.includes('.')
                        ? parseFloat(n.trim())
                        : parseInt(n.trim())) * amount * DIGITS) / DIGITS)
                .join(' ') + ' )'
        })
        // scale the texture on the brush
        newBrush = newBrush.replace(/\)\s+([^\)\(]*?)\s+((\s*[0-9\.-]+\s*){5,8})/ig, (str, $1, $2) => {
            return ') ' + $1 + ' ' + $2.trim().split(/\s+/ig)
                .map((n, i) => i >= 5 || i <= 2 ? n : (Math.round((n.includes('.')
                        ? parseFloat(n.trim())
                        : parseInt(n.trim())) * amount * DIGITS) / DIGITS))
                .join(' ') + '\n'
        })
        file = file.replace(b, newBrush)
    })
    
    // replace all origins with scaled
    // TODO: make this a function
    var origins = importer.regexToArray(/"origin"\s+"((\s*[0-9\.-]+\s*)*)"/ig, file, 1)
    origins.forEach($1 => {
        var newOrigin = $1.trim().split(/\s+/ig)
            .map(n => Math.round((n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim())) * amount * DIGITS) / DIGITS)
            .join(' ')
        file = file.replace(new RegExp('"origin"\\s+"' + $1 + '"', 'ig'), '"origin" "' + newOrigin + '"')
    })
    var lips = importer.regexToArray(/"lip"\s+"((\s*[0-9\.-]+\s*)*)"/ig, file, 1)
    lips.forEach($1 => {
        var newOrigin = $1.trim().split(/\s+/ig)
            .map(n => Math.round((n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim())) * amount * DIGITS) / DIGITS)
            .join(' ')
        file = file.replace(new RegExp('"lip"\\s+"' + $1 + '"', 'ig'), '"lip" "' + newOrigin + '"')
    })
    
    // scale models
    var models = importer.regexToArray(/\{[\s\S^\}\{]*?"classname"\s+"misc_model"[\s\S^\}\{]*?\}/ig, file)
    models.forEach(m => {
        var newModel = m
        if(m.match(/"modelscale"/i)) {
            var match = (/"modelscale"\s+"([^"]*?)"/ig).exec(m)
            var scaled = (match[1].includes('.')
                ? parseFloat(match[1])
                : parseInt(match[1])) * amount
            newModel = newModel.replace(match[0], '"modelscale" "' + scaled + '"')
        } else {
            newModel = newModel.replace(/"classname"\s+"misc_model"/ig,
                                        '"classname" "misc_model"\n "modelscale" "' + amount + '"')
        }
        file = file.replace(m, newModel)
    })
    
    return file
}

module.exports = scaleMap


#### translate quake map?



In [ ]:
var importer = require('../Core')


function translateMap(file, amount) {
    // get all brushes in map, leaf nodes with at least one vertex
    var brushes = importer.regexToArray(/\{[^\{\}]*?\}/ig, file)
    
    // replace all brushes with scaled values
    brushes.forEach(b => {
        var newBrush = b
        newBrush = newBrush.replace(/\(((\s*[0-9\.-]+\s*)*)\)/ig, (str, $1) => {
            return '( ' + $1.trim().split(/\s+/ig)
                .map((n, i) => (n.includes('.')
                        ? parseFloat(n.trim())
                        : parseInt(n.trim())) + amount[i])
                .join(' ') + ' )'
        })
        file = file.replace(b, newBrush)
    })
    
    // replace all origins with scaled
    // TODO: make this a function
    var origins = importer.regexToArray(/"origin"\s+"((\s*[0-9\.-]+\s*)*)"/ig, file, 1)
    origins.forEach($1 => {
        var newOrigin = $1.trim().split(/\s+/ig)
            .map((n, i) => (n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim())) + amount[i])
            .join(' ')
        file = file.replace(new RegExp('"origin"\\s+"' + $1 + '"', 'ig'), '"origin" "' + newOrigin + '"')
    })
    
    return file
}

module.exports = translateMap


#### replace common textures in map?


In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')

var common = [
  'areaportal',    'botclip',
  'caulk',         'clip',
  'clusterportal', 'cushion',
  'donotenter',    'full_clip',
  'hint',          'ladderclip',
  'ladderclip',    'lightgrid',
  'metalclip',     'missileclip',
  'nodraw',        'nodrawnonsolid',
  'nodrop',        'nolightmap',
  'origin',        'qer_mirror',
  'qer_portal',    'slick',
  'terrain',       'trigger',
  'weapclip',      'white'
];
var textures = {
    
}
function replaceTextures(file) {
    var used = []
    
    if(typeof file === 'string' && fs.existsSync(file)) {
        file = fs.readFileSync(file).toString('utf-8')
    }
    
    // get a list of common textures
    if(!common) {
        var files = fs.readdirSync('/Users/briancullinan/planet_quake_data/quake3-baseq3/common-spog.pk3dir/textures/common')
        common = files.map(f => f.substr(0, f.length - path.extname(f).length));
    }
    
    // get all brushes in map, leaf nodes with at least one vertex
    var brushes = importer.regexToArray(/\{[\s\S^}]*?\}/ig, file)
    
    // replace all brushes with textures from common or berserker overrides
    brushes.forEach(b => {
        var newBrush = b
        Object.keys(textures).forEach(k => {
            newBrush = newBrush.replace(new RegExp('\\) ([^\\/\\)\\(]*?\\/)*?' + k, 'ig'),
                                        (str, $1) => (') ' + textures[k]))
        })
        common.forEach(k => {
            newBrush = newBrush.replace(new RegExp('\\) ([^\\/\\)\\(]*?\\/)*?' + k, 'ig'),
                                (str, $1) => (') common/' + (k === 'clip' ? 'hint' : k)))
        })
        // fix water, don't mix textures because
        //   trenchbroom rearranges verteces so water isn't always applied
        if(newBrush.includes('wter')) {
            newBrush = newBrush
                .replace(/\)\s+([^\)\(]*?)\s+((\s*[0-9\.-]+){5,8})/igm,
                         (str, $1, $2) => (') e1u1/bluwter ' + $2))
        }
        var texts = importer.regexToArray(/\)\s+([^\)\(]*?)\s+((\s*[0-9\.-]+){5,8})/igm, b, 1)
        texts.forEach($1 => {
            if(used.indexOf(path.dirname($1)) === -1) used.push(path.dirname($1))
        })
        
        file = file.replace(b, newBrush)
    })
    
    used = used.map(u => 'textures/' + u.replace('^\/|\/$|^\s*|\s*$', ''))
    file = file.replace(/"classname" "worldspawn"/ig,`"classname" "worldspawn"
"_tb_textures" "textures/common;${used.join(';')}"`)

    return file
}

module.exports = replaceTextures


#### replace known classes in map?



In [ ]:
var fs = require('fs')

var classes = {
    weapon_chaingun:                'weapon_lightning',
    misc_teleporter:                'trigger_teleport',
    weapon_hyperblasters:           'weapon_plasmagun',
    weapon_supershotgun:            'weapon_shotgun',
    info_teleport_destination:      'target_position',
    item_armor1:                    'item_armor_combat',
    item_armor2:                    'item_armor_combat',
    item_armorInv:                  'item_armor_body',
    item_artifact_envirosuit:       'item_enviro',
    item_artifact_invisibility:     'item_invis',
    item_artifact_super_damage:     'item_quad',
    item_cells:                     'ammo_lightning',
    item_rockets:                   'ammo_rockets',
    item_shells:                    'ammo_shells',
    item_spikes:                    'ammo_cells',
    weapon_nailgun:                 'weapon_plasmagun',
    weapon_supernailgun:            'weapon_plasmagun',
    weapon_supershotgun:            'weapon_shotgun',
    // replace target_help with target_print
    target_help:                    'target_print',
    func_door_rotating:             'func_rotating'
}
function replaceClasses(file) {
    if(typeof file === 'string' && fs.existsSync(file)) {
        file = fs.readFileSync(file).toString('utf-8')
    }
    
    // replace explosions with misc_model
    file = file.replace(/"classname" "target_explosion"/ig,
                        '"classname" "misc_model"\n "model" "models/objects/r_explode/tris.md2"')
    // TODO: replace explosions with animated delay
    // replace tigger_once with multiple wait -1
    file = file.replace(/"classname" "trigger_once"/ig,
                        '"classname" "trigger_multiple"\n "wait" "-1"')
    // replace classes
    Object.keys(classes).forEach(k => {
        file = file.replace(new RegExp('"classname" "' + k + '"', 'ig'),
                            '"classname" "' + classes[k] + '"')
    })
    return file
}

module.exports = replaceClasses


#### replace entities in map?



In [ ]:
var fs = require('fs')
var importer = require('../Core')

var MATCH_MODELS = /QUAKED\s+([^\s]*)\s+[\s\S]*?model="([^"]*)"/i;

var ents;
function replaceEntities(file, entities) {
    if(!entities) {
        entities = '/Users/briancullinan/planet_quake_data/quake2-baseq2/scripts/entities.def'
    }
    
    // parse entities definition
    if(!ents) {
        var entityStr = fs.readFileSync(entities).toString('utf-8')
        var matches = importer.regexToArray(/QUAKED\s+.*/ig, entityStr)
        ents = matches.reduce((obj, m, i, arr) => {
            var start = entityStr.indexOf(m)
            var end = entityStr.indexOf(arr[i + 1])
            if(end === -1) end = entityStr.length
            var groups = MATCH_MODELS.exec(entityStr.substring(start, end))
            if(!groups) return obj
            obj[groups[1]] = groups[2]
            return obj
        }, {})
    }
    
    if(typeof file === 'string' && fs.existsSync(file)) {
        file = fs.readFileSync(file).toString('utf-8')
    }
    
    // replace entities
    file = file.replace(/"misc_([^"]*)"/ig, ($0) => `"misc_model"\n "model" ${$0}`)
    
    var noises = importer.regexToArray(/noise.*?"\s*"(.*?)"/ig, file, false)
    noises.forEach(n => {
        if(!n[1].includes('sound/')) {
            file = file.replace(n[0], `noise" "sound/${n[1]}"`)
        }
    })
    
    // replace misc_ with models from ent
    Object.keys(ents).forEach(k => {
        if(!k.includes('misc_')) return
        file = file.replace(new RegExp('"' + k + '"', 'ig'), '"' + ents[k] + '"')
    })
    return file
}

module.exports = replaceEntities;


#### add a skybox and remove sky entities?

add skybox to map?

Because quake 2 maps leak somewhere during this process.


In [ ]:
var importer = require('../Core')
var {doIntersect} = importer.import('brush to vertex')

function getBounds(file) {
    // get all brushes in map, leaf nodes with at least one vertex
    var brushes = importer.regexToArray(/\{[\s\S]*?\}/ig, file)
   
    brushes = brushes.map(b => {
        var points = importer
            .regexToArray(/\(((\s*[0-9\.-]+\s*)*)\)/ig, b, 1)
            .map(m => m.trim().split(/\s+/ig)
                .map(n => (n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim()))))
        
        return [[
            Math.min.apply(null, points.map(b => b[0])),
            Math.min.apply(null, points.map(b => b[1])),
            Math.min.apply(null, points.map(b => b[2]))
        ], [
            Math.max.apply(null, points.map(b => b[0])),
            Math.max.apply(null, points.map(b => b[1])),
            Math.max.apply(null, points.map(b => b[2]))
        ]]
    })
 
    // replace all origins with scaled
    // TODO: make this a function
    var origins = importer
        .regexToArray(/"origin"\s+"((\s*[0-9\.-]+\s*)*)"/ig, file, 1)
        .map(o => o.trim().split(/\s+/ig)
            .map(n => (n.includes('.')
                ? parseFloat(n.trim())
                : parseInt(n.trim()))))
    
    origins = origins.concat.apply(origins, brushes)
        .filter(o => o && isFinite(o[0]))
    
    return [[
        Math.min.apply(null, origins.map(b => b[0])),
        Math.min.apply(null, origins.map(b => b[1])),
        Math.min.apply(null, origins.map(b => b[2]))
    ], [
        Math.max.apply(null, origins.map(b => b[0])),
        Math.max.apply(null, origins.map(b => b[1])),
        Math.max.apply(null, origins.map(b => b[2]))
    ]]
}


function addSkybox(fileName) {
    var file
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        file = fs.readFileSync(fileName).toString('utf-8')
    } else {
        file = fileName
    }

    var brushes = importer.regexToArray(/\{[^\{}]*?\}\s*/ig, file)
    brushes.forEach(b => {
        if(b.includes('/sky')) {
            file = file.replace(b, '')
            return false
        }
        return true
    })
    
    var vs = getBounds(file)
    
    // TODO: use a fancy for loop instead contains each corner and extends towards the next two points?
    var points = [
        [vs[0][0], vs[0][1], vs[0][2]-16],
        [vs[1][0], vs[1][1], vs[0][2]],
        
        [vs[0][0]-16, vs[0][1], vs[0][2]],
        [vs[0][0],    vs[1][1], vs[1][2]],
        
        [vs[0][0], vs[0][1]-16, vs[0][2]],
        [vs[1][0], vs[0][1],    vs[1][2]],
        
        
        [vs[0][0], vs[0][1], vs[1][2]],
        [vs[1][0], vs[1][1], vs[1][2]+16],
        
        [vs[1][0],    vs[0][1], vs[0][2]],
        [vs[1][0]+16, vs[1][1], vs[1][2]],
        
        [vs[0][0], vs[1][1],    vs[0][2]],
        [vs[1][0], vs[1][1]+16, vs[1][2]],
        
    ]
    var newBrush = ``
    for(var i = 0; i < points.length / 2; i++) {
        var p1 = points[i*2]
        var p2 = points[i*2+1]
        newBrush += `
{ // brush 0
( ${p1[0]} ${p1[1]} ${p2[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) ( ${p1[0]} ${p2[1]} ${p1[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
( ${p2[0]} ${p2[1]} ${p2[2]} ) ( ${p2[0]} ${p2[1]} ${p1[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
( ${p2[0]} ${p1[1]} ${p2[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
( ${p1[0]} ${p2[1]} ${p2[2]} ) ( ${p1[0]} ${p2[1]} ${p1[2]} ) ( ${p2[0]} ${p2[1]} ${p1[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
( ${p1[0]} ${p2[1]} ${p1[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
( ${p1[0]} ${p1[1]} ${p2[2]} ) ( ${p1[0]} ${p2[1]} ${p2[2]} ) ( ${p2[0]} ${p2[1]} ${p2[2]} ) e1u1/sky1 0 0 0 1 1 0 0 0
}
`
    }
    

    var exp = (/\{*\s*\/\/\s*brush\s*0\s*\{*/ig)
    var match = exp.exec(file)
    var pos = exp.lastIndex
    file = file.substr(0, pos - match[0].length) + newBrush + file.substr(pos - match[0].length)
    
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        console.log(`writing ${fileName}`)
        fs.writeFileSync(fileName, file)
    } else {
        return file
    }
}

module.exports = {
    addSkybox,
    getBounds
}



#### draw hints from x and y coordinates?

draw hints in map?



In [ ]:
var path = require('path')
var importer = require('../Core')
var {getBounds} = importer.import('add skybox to map')


var MAPS_HINTS = {
    'base1-a': [
        [-648, -209],
        [-648, 235],
        [-400, 235],
        [-400, 514],
        [300, 514],
        [300, 1922],
        [9999, 1922],
        [9999, -9999],
        [-648, -9999],
        [-648, -209]
    ],
    'base1-b': [
        [-648, -209],
        [-648, 235],
        [-400, 235],
        [-400, 514],
        [300, 514],
        [300, 1922],
        [-1300, 2174],
        [-1300, 648],
        [-648, -209],
    ],
    'base1-c': [
        [-1300, 1000],
        [-1300, 2174],
        [-9999, 9999],
        [-9999, 1000],
        [-1300, 1000],
    ],
    
    'base2-a': [
        [-1212, 1242],
        [340, 1242],
        [340, 1030],
        [1324, 1030],
        [1324, 3109],
        [-1212, 3109],
        [-1212, 1242]
    ],
    'base2-b': [
        [-1212, 1242],
        [340, 1242],
        [340, 1030],
        [1324, 1030],
        [1324, -1711],
        [736, -1711],
        [736, -1251],
        [-1212, -1251],
        [-1212, 1242]
    ],
    'base2-c': [
        [1324, 1030],
        [1324, -1711],
        [736, -1711],
        [736, -1251],
        [-1212, -1251],
        [-1212, -2815],
        [1324, -2815],
        [1324, 1030],
    ]
}

function addHints(fileName, hints) {
    var file
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        file = fs.readFileSync(fileName).toString('utf-8')
        if(!hints) {
            hints = path.basename(fileName).replace(/[-_]converted|\.map$/ig, '')
        }
    } else {
        file = fileName
    }
    
    if(!Array.isArray(hints)) {
        var exp = new RegExp(hints, 'ig')
        hints = Object.keys(MAPS_HINTS).reduce((arr, k) => {
            if(k.match(exp)) {
                arr.push(MAPS_HINTS[k])
            }
            return arr
        }, [])
    }

    var brushes = importer.regexToArray(/\{[^\{}]*?\}\s*/ig, file)
    
    /*
    brushes.forEach(b => {
        if(b.includes('/areaportal')) {
            file = file.replace(b, '')
            return false
        }
        return true
    })
    */
    
    var vs = getBounds(file)
    
    var newBrush = ``
    
    for(var h = 0; h < hints.length; h++) {
        for(var i = 0; i < hints[h].length - 1; i++) {
            var minX = Math.min(hints[h][i][0], hints[h][i+1][0]), 
                maxX = Math.max(hints[h][i][0], hints[h][i+1][0]), 
                minY = Math.min(hints[h][i][1], hints[h][i+1][1]), 
                maxY = Math.max(hints[h][i][1], hints[h][i+1][1])
            var p1 = [minX,   minY, vs[0][2]]
            var p2 = [maxX+2, maxY+2, vs[1][2]]

            newBrush += `
{ // brush 0
( ${p1[0]} ${p1[1]} ${p2[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) ( ${p1[0]} ${p2[1]} ${p1[2]} ) common/hint 0 0 0 1 1 0 0 0
( ${p2[0]} ${p2[1]} ${p2[2]} ) ( ${p2[0]} ${p2[1]} ${p1[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) common/hint 0 0 0 1 1 0 0 0
( ${p2[0]} ${p1[1]} ${p2[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) common/hint 0 0 0 1 1 0 0 0
( ${p1[0]} ${p2[1]} ${p2[2]} ) ( ${p1[0]} ${p2[1]} ${p1[2]} ) ( ${p2[0]} ${p2[1]} ${p1[2]} ) common/hint 0 0 0 1 1 0 0 0
( ${p1[0]} ${p2[1]} ${p1[2]} ) ( ${p1[0]} ${p1[1]} ${p1[2]} ) ( ${p2[0]} ${p1[1]} ${p1[2]} ) common/hint 0 0 0 1 1 0 0 0
( ${p1[0]} ${p1[1]} ${p2[2]} ) ( ${p1[0]} ${p2[1]} ${p2[2]} ) ( ${p2[0]} ${p2[1]} ${p2[2]} ) common/hint 0 0 0 1 1 0 0 0
}
`
        }
    }
    
    var exp = (/\{*\s*\/\/\s*brush\s*0\s*\{*/ig)
    var match = exp.exec(file)
    var pos = exp.lastIndex
    file = file.substr(0, pos - match[0].length) + newBrush + file.substr(pos - match[0].length)
    
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        console.log(`writing ${fileName}`)
        fs.writeFileSync(fileName, file)
    } else {
        return file
    }
}

module.exports = addHints
module.exports.addHints = addHints
module.exports.MAPS_HINTS = MAPS_HINTS


#### split map using hints and portals?

copy brushes inside a polygon?

split map hints?


In [ ]:
var path = require('path')
var importer = require('../Core')
var {MAPS_HINTS, addHints} = importer.import('draw hints in map')
var translateMap = importer.import('translate quake map')
var {isInside} = importer.import('brush to vertex')
var {addSkybox} = importer.import('add skybox to map')

var BUFFER_UNITS = 200
var ENTITIES_EXP = /\{[^\{\}]*?"classname"\s*"([^"]*)"[^\{\}]*?(\{[^\{\}]*?\}[^\{\}]*?)*\}/ig
var BRUSH_EXP = /\{[^\{\}]*?\}\s*/ig
var moves = [
    [0, 0],
    [BUFFER_UNITS, 0],
    [0, BUFFER_UNITS],
    [BUFFER_UNITS, BUFFER_UNITS],
    [-BUFFER_UNITS, 0],
    [0, -BUFFER_UNITS],
    [-BUFFER_UNITS, -BUFFER_UNITS],
]
function splitHints(fileName, hints) {
    var file
    if(typeof fileName === 'string' && fs.existsSync(fileName)) {
        file = fs.readFileSync(fileName).toString('utf-8')
        if(!hints) {
            hints = path.basename(fileName).replace(/[-_]converted|\.map$/ig, '')
        }
    } else {
        file = fileName
    }
    
    if(typeof hints !== 'object') {
        var exp = new RegExp(hints, 'ig')
        hints = Object.keys(MAPS_HINTS).reduce((obj, k) => {
            if(k.match(exp)) {
                obj[k] = MAPS_HINTS[k]
            }
            return obj
        }, {})
    }
    
    var multimaps = {}
    var worldspawn = {}
    Object.keys(hints).forEach(k => (multimaps[k] = [], worldspawn[k] = []))
    
    var entities = importer
        .regexToArray(ENTITIES_EXP, file, false)
    
    entities.forEach(ent => {
        var include = false
        var points = []
        ent.points = points
        
        var brushes = importer.regexToArray(BRUSH_EXP, ent[0])

        brushes.forEach(brush => {
            var pts = importer
                .regexToArray(/\(((\s*[0-9\.-]+\s*)*)\)/ig, brush, 1)
                .map(m => m.trim().split(/\s+/ig)
                    .map(n => (n.includes('.')
                        ? parseFloat(n.trim())
                        : parseInt(n.trim()))))
            
            if(brush.includes('/sky')) {
                return
            }

            pts.forEach(b => {
                Object.keys(hints).forEach(k => {
                    if(worldspawn[k].indexOf(brush) === -1) {
                        // if moving the point in any direction would
                        //   cause it to be inside, include that brush
                        for(var m = 0; m < moves.length; m++) {
                            if(isInside(hints[k].map(h => ({x: h[0], y: h[1]})), 
                                        hints[k].length-1, 
                                        {x: b[0] + moves[m][0], y: b[1] + moves[m][1]})) {
                                points.push.apply(pts)
                                worldspawn[k].push(brush) // object reference points added below
                                return
                            }
                        }
                    }
                })
            })
        })

        var skipOrigin = false
        Object.keys(worldspawn).forEach(k => {
            if(!worldspawn[k].length) return

            multimaps[k].push({
                '0': ent[0].replace(BRUSH_EXP, '').replace('}', worldspawn[k].join('\n') + '}'),
                points: points
            })
            worldspawn[k] = []
            skipOrigin = true
        })
        if(skipOrigin) return // already know its a brush type not point type

        var origins = importer
            .regexToArray(/"origin"\s+"((\s*[0-9\.-]+\s*)*)"/ig, ent[0], 1)
            .map(o => o.trim().split(/\s+/ig)
                .map(n => (n.includes('.')
                    ? parseFloat(n.trim())
                    : parseInt(n.trim()))))
        
        origins.forEach(b => {
            Object.keys(hints).forEach(k => {
                // if moving the point in any direction would
                //   cause it to be inside, include that brush
                for(var m = 0; m < moves.length; m++) {
                    if(isInside(hints[k].map(h => ({x: h[0], y: h[1]})), 
                                hints[k].length-1, 
                                {x: b[0] + moves[m][0], y: b[1] + moves[m][1]})) {
                        points.push(b)
                        multimaps[k].push(ent)
                    }
                }
            })
        })
    })
    
    // get middle 50% of all points
    var translations = Object.keys(multimaps).map(k => {
        var result = []
        for(var i = 0; i < 3; i++) {
            var points = [].concat.apply([], multimaps[k]
                .map(ent => ent.points.map(p => p[i])))
            points.sort()
            result[i] = -Math.round((points
                // get the middle half
                .slice(points.length / 4, points.length / 4 * 3)
                // get the average
                .reduce((a, b) => a + b, 0) / (points.length / 2))
                // round to the nearest grid position?
                / 147.2) * 147.2
        }
        return result
    })
    
    console.log(translations)
    
    // copy all the brushes in to a new file, skybox and save
    var maps = Object.keys(multimaps).forEach((k, i) => {
        var map = multimaps[k]
            .filter((ent, i, arr) => arr.indexOf(ent) === i)
            .map(ent => ent[0])
            .join('\n')
        
        map = translateMap(map, translations[i])
        
        map = addSkybox(map)
        
        file = addHints(file, [hints[k]])
        
        if(typeof fileName === 'string' && fs.existsSync(fileName)) {
            var newFile = path.join(path.dirname(path.resolve(fileName)), path.basename(k) + '.map')
            console.log(`writing ${newFile}`)
            fs.writeFileSync(newFile, map)
        } else {
            return map
        }
    })
    
    return maps
}

module.exports = splitHints


#### TODO: convert quake 2 entities to quake 3?


TODO: use "bodyque" for dead body model explosions? these have a high health in quake 2, they should drop the goods.





```
Quake 1:
------------------------------------------------
func_plat                      *
func_door                      *
info_null                      *
info_player_deathmatch         *
item_health                    *
info_teleport_destination      target_position
item_armor1                    item_armor_combat
item_armor2                    item_armor_combat
item_armorInv                  item_armor_body
item_artifact_envirosuit       item_enviro
item_artifact_invisibility     item_invis
item_artifact_super_damage     item_quad
item_cells                     ammo_lightning
item_rockets                   ammo_rockets
item_shells                    ammo_shells
item_spikes                    ammo_cells
weapon_nailgun                 weapon_plasmagun
weapon_supernailgun            weapon_plasmagun
weapon_supershotgun            weapon_shotgun
weapon_rocketlauncher          *
light                          *
trigger_hurt                   *
trigger_multiple               *
trigger_push                   *
trigger_teleport               *
weapon_grenadelauncher         *
weapon_lightning               *

Quake 2:
-------------------------------------------------
ammo_bullets               *
ammo_cells                 *
ammo_grenades              *
ammo_rockets               *
ammo_shells                *
ammo_slugs                 *
func_button                *
func_door                  *
func_group                 *
func_plat                  *
func_rotating              *
func_timer                 *
func_train                 *
info_notnull               *
info_null                  *
info_player_deathmatch     *
info_player_intermission   *
item_armor_body            *
item_armor_combat          *
item_armor_shard           *
item_enviro                *
item_health                *
item_health_large          *
item_health_mega           *
item_health_small          *
item_quad                  *
light                      *
misc_teleporter_dest       *
path_corner                *
target_laser               *
target_speaker             *
trigger_always             *
trigger_hurt               *
trigger_multiple           *
trigger_push               *
weapon_bfg                 *
weapon_chaingun            weapon_lightning
misc_teleporter            trigger_teleport 
weapon_hyperblasters       weapon_plasmagun
weapon_supershotgun        weapon_shotgun
weapon_grenadelauncher     *
weapon_machinegun          *
weapon_railgun             *
weapon_rocketlauncher      *
weapon_shotgun             *
```


## informational



### misc map/bsp scripts



#### get vertex from brush strokes?

brush to vertex?


In [ ]:
  
// Given three colinear points p, q, r, the function checks if 
// point q lies on line segment 'pr' 
function onSegment(p, q, r) 
{
    if (q.x <= Math.max(p.x, r.x) && q.x >= Math.min(p.x, r.x) && 
            q.y <= Math.max(p.y, r.y) && q.y >= Math.min(p.y, r.y)) {
        return true; 
    }
    return false; 
} 
  
// To find orientation of ordered triplet (p, q, r). 
// The function returns following values 
// 0 --> p, q and r are colinear 
// 1 --> Clockwise 
// 2 --> Counterclockwise 
function orientation(p, q, r) 
{ 
    var val = (q.y - p.y) * (r.x - q.x) - 
              (q.x - p.x) * (r.y - q.y); 
  
    if (val == 0) return 0;  // colinear 
    return (val > 0)? 1: 2; // clock or counterclock wise 
} 
  
// The function that returns true if line segment 'p1q1' 
// and 'p2q2' intersect. 
function doIntersect(p1, q1, p2, q2) 
{ 
    // Find the four orientations needed for general and 
    // special cases 
    var o1 = orientation(p1, q1, p2); 
    var o2 = orientation(p1, q1, q2); 
    var o3 = orientation(p2, q2, p1); 
    var o4 = orientation(p2, q2, q1); 
  
    // General case 
    if (o1 != o2 && o3 != o4) {
        return true
    }
  
    // Special Cases 
    // p1, q1 and p2 are colinear and p2 lies on segment p1q1 
    if (o1 == 0 && onSegment(p1, p2, q1)) return true
  
    // p1, q1 and p2 are colinear and q2 lies on segment p1q1 
    if (o2 == 0 && onSegment(p1, q2, q1)) return true
  
    // p2, q2 and p1 are colinear and p1 lies on segment p2q2 
    if (o3 == 0 && onSegment(p2, p1, q2)) return true
  
     // p2, q2 and q1 are colinear and q1 lies on segment p2q2 
    if (o4 == 0 && onSegment(p2, q1, q2)) return true
  
    return false; // Doesn't fall in any of the above cases 
} 


// Returns true if the point p lies inside the polygon[] with n vertices 
function isInside(polygon, n, p) 
{ 
    // There must be at least 3 vertices in polygon[] 
    if (n < 3)  return false; 
  
    // Create a point for line segment from p to infinite 
    var extreme = {x: Number.MAX_VALUE, y: p.y}; 
  
    // Count intersections of the above line with sides of polygon 
    var count = 0, i = 0; 
    do
    { 
        var next = (i+1)%n; 
  
        // Check if the line segment from 'p' to 'extreme' intersects 
        // with the line segment from 'polygon[i]' to 'polygon[next]' 
        if (doIntersect(polygon[i], polygon[next], p, extreme)) 
        { 
            // If the point 'p' is colinear with line segment 'i-next', 
            // then check if it lies on segment. If it lies, return true, 
            // otherwise false 
            if (orientation(polygon[i], p, polygon[next]) == 0) 
               return onSegment(polygon[i], p, polygon[next]); 
  
            count++; 
        } 
        i = next; 
    } while (i != 0); 
  
    // Return true if count is odd, false otherwise 
    return (count%2 == 1);
} 


module.exports = {
    doIntersect,
    isInside
}


#### TODO: 



#### list noises in a quake 3 map?



In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')

function listNoise() {
    var contents = fs.readFileSync('/Users/briancullinan/.q3a/baseq3/baseq2.pk3dir/maps/base1.map').toString('utf8')
    var noises = importer.regexToArray(/noise.*?"\s*"(.*?)"/ig, contents, 1)
    return {all: noises, missing: noises.filter(n => {
        return !fs.existsSync(path.join('/Users/briancullinan/.q3a/baseq3/baseq2.pk3dir/sound/', n))
    })}
}

module.exports = listNoise

#### list textures in quake 3 map?



In [ ]:
var fs = require('fs')
var path = require('path')

function listTextures(file, dirsOnly) {
    if(typeof file === 'string' && fs.existsSync(file)) {
        file = fs.readFileSync(file).toString('utf-8')
    }
    
    var used = []
    var brushes = importer.regexToArray(/\{[\s\S^\}\{]*?\}/ig, file, -1) // returns indexes
    console.log(`found ${brushes.length} brushes.`)
    for(var i = 0; i < brushes.length; i++) {
        var b = file.substr(brushes[i][0], brushes[i][1])
        var texts = importer.regexToArray(/\)\s+([^\)\(]*?)\s+((\s*[0-9\.-]+){5,8})/igm, b, 1)
        texts.forEach($1 => {
            if(dirsOnly) {
                if(!used.includes(path.dirname($1)))
                    used.push(path.dirname($1))
            } else {
                if(!used.includes($1))
                    used.push($1)
            }
        })
    }
    return used
}

module.exports = listTextures



#### list shaders in quake 3 shader?



In [ ]:
var importer = require('../Core')
var balanced = importer.import('balanced')

function findShaders(file) {
    if(typeof file === 'string' && fs.existsSync(file)) {
        file = fs.readFileSync(file).toString('utf-8')
            .replace(/\/\/.*/ig, '')
    }
    var match
    var current = file
    var result = {}
    while((match = balanced('{', '}', current))) {
        var name = match.pre.trim()
        console.log('found shader ' + name)
        result[name] = match.body
        current = match.post
    }
    return result
}

module.exports = findShaders


#### find all shaders?



In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var findShaders = importer.import('list shaders in quake 3 shader')

async function findAllShaders(modDir) {
    var allShaders = {}
    var shaders = await glob('**/*.shader', modDir)
    for(var i = 0; i < shaders.length; i++) {
        console.log(shaders[i])
        Object.assign(allShaders, findShaders(shaders[i]))
    }
    fs.writeFileSync(path.join(modDir, 'all_shaders.json'),
                     JSON.stringify(allShaders, null, 2))
    return Object.keys(allShaders)
}

module.exports = findAllShaders


#### search textures directory for paths?




In [ ]:
var path = require('path')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var {chext, chroot} = importer.import('changing file name')
var listTextures = importer.import('list textures in quake 3 map')
var mkdirpSync = importer.import('mkdirp')
//var findShaders = importer.import('list shaders in quake 3 shader')

async function findTextures(file, modDir, outDir) {
    var textures = listTextures(file)
    var shaders = require(path.join(modDir, 'all_shaders.json'))
    
    var shaded = []
    for(var i = 0; i < textures.length; i++) {
        var realTexture = 'textures/' + textures[i]
        if(typeof shaders[realTexture] != 'undefined') {
            console.log('shaded ' + realTexture)
            shaded.push(textures[i])
            var shader = shaders[realTexture]
            var extraTextures = importer
                .regexToArray(/map\s([^\s]+?\/[^\s]+?$)/img, shader, 1)
                .map(t => chext(t, ''))
            textures.push.apply(textures, extraTextures)
        }
    }
    
    var defaults = textures
        .filter(t => !shaded.includes(t))
    var matched = await glob(defaults.map(t => '**/' + t + '*'), modDir)
    var listStr = matched.join(';')
    var unmatched = defaults
        .filter(t => !listStr.includes(path.basename(t)))
    
    if(outDir) {
        for(var i = 0; i < matched.length; i++) {
            var outFile = chroot(matched[i], modDir, outDir)
            mkdirpSync(path.dirname(outFile))
            fs.copyFileSync(matched[i], outFile)
        }
    }
    
    return {
        matched: matched,
        unmatched: unmatched
    }
}

module.exports = findTextures


### QVM analysis




#### disassemble all QVMs?



In [ ]:
var fs = require('fs')
var path = require('path')
var importer = require('../Core')
var {chext} = importer.import('changing file name')
var exec = importer.import('spawn child process')
var {glob} = importer.import('glob files')

var PROJECT = '/Users/briancullinan/planet_quake_data/'
var QVM_DIS = '/Users/briancullinan/planet_quake_data/tools/q3vm-master/qvmdis'

async function disassembleQVMs(project) {
    if(!project) project = PROJECT
    var qvms = await glob('**/*.qvm', project)
    console.log(`Found ${qvms.length} QVMs`)
    for(var i = 0; i < qvms.length; i++) {
        console.log(`Disassembling ${qvms[i]}`)
        var type
        if(qvms[i].match(/cgame/i)) type = 'cgame'
        else if(qvms[i].match(/qagame/i)) type = 'game'
        else if(qvms[i].match(/ui/i)) type = 'ui'
        else throw new Error(`Don't know what kind of QVM ${qvms[i]} is.`)
        var output = chext(qvms[i], '.dis')
        if(fs.existsSync(output)) continue
        try {
            await exec(`${QVM_DIS} "${qvms[i]}" ${type} > "${output}"`)
        } catch (e) {
            console.log(e)
        }
    }
}

module.exports = disassembleQVMs
